In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

# Define dataset path (adjust spacing carefully)
data_path = "/content/drive/MyDrive/Capstone project/Dataset"
data_path_saved_model = "/content/drive/MyDrive/Capstone project"

Mounted at /content/drive


In [ ]:
!pip install fasttext

In [ ]:
import fasttext
import os

In [ ]:
train_file = os.path.join(data_path, "train.ft.txt")
test_file = os.path.join(data_path, "test.ft.txt")

print("Train file:", train_file)
print("Test file:", test_file)

Train file: /content/drive/MyDrive/Capstone project/Dataset/train.ft.txt
Test file: /content/drive/MyDrive/Capstone project/Dataset/test.ft.txt


In [ ]:
# Show first 10 lines from the training data
with open(train_file, 'r', encoding='utf-8') as f:
    for i in range(10):
        print(f.readline().strip())

print("\n--- TEST FILE PREVIEW ---\n")

# Show first 10 lines from the test data
with open(test_file, 'r', encoding='utf-8') as f:
    for i in range(10):
        print(f.readline().strip())


__label__2 Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
__label__2 The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.
__label__2 Amazing!: This soundtrack is m

In [ ]:
model = fasttext.train_supervised(
    input=train_file,
    lr=0.1,            # learning rate
    epoch=10,          # number of epochs
    wordNgrams=2,      # include word bigrams
    dim=100,           # embedding dimension
    loss='softmax',    # loss function
    thread=8           # parallel threads
)

In [ ]:
model.save_model(os.path.join(data_path_saved_model, "model_amzn.bin"))

In [ ]:
def print_results(N, p, r):
    print(f"Total Samples (N): {N}")
    print(f"Precision@1: {p:.3f}")
    print(f"Recall@1: {r:.3f}")
    print(f"Accuracy: {(p * 100):.2f}%")

results = model.test(test_file)
print_results(*results)

Total Samples (N): 400000
Precision@1: 0.932
Recall@1: 0.932
Accuracy: 93.15%


In [ ]:
label_metrics = model.test_label(test_file)

for label, metrics in label_metrics.items():
    print(f"{label}: {metrics}")

__label__2: {'precision': 0.931638593262383, 'recall': 0.931415, 'f1score': 0.9315267832139856}
__label__1: {'precision': 0.9314314564504519, 'recall': 0.931655, 'f1score': 0.9315432148142223}


In [ ]:
samples = [
    "This product is amazing! Works perfectly.",
    "Terrible experience, complete waste of money.",
    "Not bad but could be better."
]

labels, probs = model.predict(samples)

In [ ]:
for i in range(len(samples)):
    print(f"\nReview: {samples[i]}")
    print(f"Predicted Label: {labels[i][0]} | Probability: {probs[i][0]:.4f}")



Review: This product is amazing! Works perfectly.
Predicted Label: __label__2 | Probability: 1.0000

Review: Terrible experience, complete waste of money.
Predicted Label: __label__1 | Probability: 1.0000

Review: Not bad but could be better.
Predicted Label: __label__2 | Probability: 1.0000

Review: flabbergasted.
Predicted Label: __label__1 | Probability: 0.9953


In [ ]:
model.quantize(input=train_file, retrain=True)
model.save_model(os.path.join(data_path_saved_model, "model_amzn_compressed.ftz"))

In [ ]:
label_metrics = model.test_label(test_file)

for label, metrics in label_metrics.items():
    print(f"{label}: {metrics}")

__label__2: {'precision': 0.9286647126849304, 'recall': 0.931525, 'f1score': 0.930092657307746}
__label__1: {'precision': 0.9313134454118686, 'recall': 0.928445, 'f1score': 0.9298770105963183}
